In [1]:
from hig_data.coco import COCOStuffDataset

img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
dataset = COCOStuffDataset(img_path, mask_path, 32)


In [ ]:
from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
import torch
img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
data = coco_graph[4]
data

In [1]:
from hig_data.coco import COCOStuffGraphPrecomputedDataset
import torch
path = '/home/rfsm2/rds/hpc-work/coco/coco_graph_train'
coco_graph = COCOStuffGraphPrecomputedDataset(path)
data = coco_graph[4]
data

Found 118287 complete datapoint in /home/rfsm2/rds/hpc-work/coco/coco_graph_train


RelaxedHeteroData(
  image=[1, 8, 32, 32],
  mask_path='/home/rfsm2/rds/hpc-work/coco/coco_graph_train/00047078_mask.npy',
  image_node={
    x=[1024, 1],
    pos=[1024, 2],
  },
  class_node={
    x=[6, 183],
    pos=[6, 2],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 30] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 915] }
)

In [ ]:
coco_graph[10]

In [9]:
from hig_data.coco import CocoStuffGraphDataset
import numpy as np
import os
def create_graph_dataset_and_export(output_dir='/Users/rupertmenneer/Downloads/coco_graph'):

    img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
    mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
    coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)

    for idx in range(len(coco_graph)):
        image, mask = coco_graph.dataset[idx]
        graph = coco_graph[idx]

        name = coco_graph.dataset._fnames['image'][coco_graph.dataset._raw_idx[idx]]
        raw_id = os.path.splitext(os.path.basename(name))[0].split('-')[-1]

        np.save(os.path.join(output_dir, f'{raw_id}_image.npy'), image)
        np.save(os.path.join(output_dir, f'{raw_id}_mask.npy'), mask)
        np.savez(os.path.join(output_dir, f'{raw_id}_graph'),
                 class_node=graph['class_node'].x,
                 class_pos=graph['class_node'].pos,
                 class_edge=graph['class_node', 'class_edge', 'class_node'].edge_index,
                 class_to_image=graph['class_node', 'class_to_image', 'image_node'].edge_index,
                 )

create_graph_dataset_and_export()


In [ ]:
# from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
# import torch
# img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
# mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
# coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
# data = coco_graph[4]
# data

In [ ]:
# from hig_data.visualisation import convert_latents_to_pixels
# import PIL
# import numpy as np
# print(data.image.shape)
# img = convert_latents_to_pixels(data.image)

# PIL.Image.fromarray(img)

In [2]:
from torch_geometric.loader import DataLoader as GeoDataLoader
dls = GeoDataLoader(coco_graph, batch_size=1024, shuffle=True)
graph_batch = next(iter(dls))
graph_batch

graph_batch['image_node'].pos, graph_batch['class_node'].pos

(tensor([[  0.,   0.],
         [  8.,   0.],
         [ 16.,   0.],
         ...,
         [232., 248.],
         [240., 248.],
         [248., 248.]]),
 tensor([[123.1362, 137.9477],
         [112.9163, 112.6696],
         [  0.0000,   0.0000],
         ...,
         [229.3333,  89.3333],
         [164.6629, 183.6800],
         [126.7200,  37.1200]]))

In [3]:
for i, (batch) in enumerate(dls):
    print(i, graph_batch.image.shape)

0 torch.Size([1024, 8, 32, 32])
1 torch.Size([1024, 8, 32, 32])
2 torch.Size([1024, 8, 32, 32])
3 torch.Size([1024, 8, 32, 32])


In [ ]:
graph_batch['image_node'].pos, graph_batch['class_node'].pos

In [ ]:
# from hig_data.visualisation import save_image_batch_list
# print(graph_batch.image.shape)
# np_img = graph_batch.image[:, :3].cpu().numpy().transpose(0, 2, 3, 1)
# save_image_batch_list([np_img,np_img], row_labels=['img', 'img'], sample_batch_size=8)

In [ ]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images
# logging_generate_sample_vis(graph_batch, resize_mask=True)
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder()
img, decoded = visualise_het_graph_on_image_batch(graph_batch, vae=vae,)
print(img.shape)
plot_array_images(img)


In [ ]:
plot_array_images(decoded)

In [ ]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

In [ ]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape